In [1]:
import os
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

from sklearn.metrics import classification_report

from sklearn import model_selection, naive_bayes, svm

from sklearn.metrics import accuracy_score, recall_score, precision_score

from sklearn.model_selection import train_test_split

# Extrating data from  tar file into local repository

In [ ]:
my_tar=tarfile.open('aclImdb_v1.tar.gz')
my_tar.extractall('D:/intern_task3')
my_tar.close()

# reading the train folders from the local device

In [2]:
%%time
docs=[]
labels=[]
labels_index={}

PATH=os.getcwd()

text_data_directory=os.path.join(PATH,"D:/intern_task3/IMDB Dataset/aclImdb/train")
for name in os.listdir(text_data_directory):
    path=os.path.join(text_data_directory,name)
    if os.path.isdir(path):
        label_id=len(labels_index)
        labels_index[label_id]=name
        for fname in sorted(os.listdir(path)):
            fpath=os.path.join(path,fname)
            f=open(fpath,encoding="ISO-8859-1")
            t=f.read()
            docs.append(t)
            f.close()
            labels.append(label_id) 
    
                
print('found %s docs.'%len(docs))                                 
    
                    
                                 
                             
                                 
                                 
                                 
                                 
                                 


found 25000 docs.
Wall time: 12.6 s


In [3]:
%%time
Docs=[]
Labels=[]
Labels_index={}

PATH=os.getcwd()

text_data_directory=os.path.join(PATH,"D:/intern_task3/IMDB Dataset/aclImdb/test")
for name in os.listdir(text_data_directory):
    path=os.path.join(text_data_directory,name)
    if os.path.isdir(path):
        Label_id=len(Labels_index)
        Labels_index[Label_id]=name
        for fname in sorted(os.listdir(path)):
            fpath=os.path.join(path,fname)
            f=open(fpath,encoding="ISO-8859-1")
            t=f.read()
            Docs.append(t)
            f.close()
            Labels.append(Label_id) 
    
                
print('found %s Docs_test.'%len(Docs))                                 
    
                    
                                 
                             
                                 
                                 
                                 
                                 
                                 


found 25000 Docs_test.
Wall time: 12.4 s


## labels of train data are

In [4]:
labels_index

{0: 'neg', 1: 'pos'}

In [5]:
imdb_train=pd.DataFrame(docs)


In [6]:
imdb_train.head()

,0
0,Story of a man who has unnatural feelings for ...
1,Airport '77 starts as a brand new luxury 747 p...
2,This film lacked something I couldn't put my f...
3,"Sorry everyone,,, I know this is supposed to b..."
4,When I was little my parents took me along to ...


## test and train have the same labels soo there is no extra level in the target

In [7]:
Labels_index

{0: 'neg', 1: 'pos'}

In [8]:
imdb_test=pd.DataFrame(Docs)


In [9]:
imdb_test.head()

,0
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."
3,Not even the Beatles could write songs everyon...
4,Brass pictures (movies is not a fitting word f...


In [10]:
imdb_train.columns = ['text_train']


In [11]:
imdb_train.head()

,text_train
0,Story of a man who has unnatural feelings for ...
1,Airport '77 starts as a brand new luxury 747 p...
2,This film lacked something I couldn't put my f...
3,"Sorry everyone,,, I know this is supposed to b..."
4,When I was little my parents took me along to ...


In [12]:
imdb_test.columns=['text_test']

In [13]:
imdb_test.head()

,text_test
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."
3,Not even the Beatles could write songs everyon...
4,Brass pictures (movies is not a fitting word f...


# tokenization of columns

In [14]:
imdb_train['text_train'] = [word_tokenize(entry) for entry in imdb_train['text_train']]

print(imdb_train['text_train'].head())

0    [Story, of, a, man, who, has, unnatural, feeli...
1    [Airport, '77, starts, as, a, brand, new, luxu...
2    [This, film, lacked, something, I, could, n't,...
3    [Sorry, everyone, ,, ,, ,, I, know, this, is, ...
4    [When, I, was, little, my, parents, took, me, ...
Name: text_train, dtype: object


In [15]:
imdb_train.isnull().sum()

text_train    0
dtype: int64

# Remove Stop words, Numeric and perfom Word Stemming/Lemmatization.

In [16]:
# Initializing WordNetLemmatizer()
word_Lemmatized = WordNetLemmatizer()

In [17]:
for index, entry in enumerate(imdb_train['text_train']):
    #print("entry:",entry)
    
    # Declaring an Empty List to store the words that follow the rules for this step
    Final_words = []
    
    for word in entry:
        #print("word:",word)
        
        
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word)
            Final_words.append(word_Final)
            
    # The final processed set of words for each iteration will be stored in 'text_train'
    imdb_train.loc[index,'text_final_train'] = str(Final_words) 

In [18]:
imdb_train.head()

,text_train,text_final_train
0,"[Story, of, a, man, who, has, unnatural, feeli...","['Story', 'man', 'unnatural', 'feeling', 'pig'..."
1,"[Airport, '77, starts, as, a, brand, new, luxu...","['Airport', 'start', 'brand', 'new', 'luxury',..."
2,"[This, film, lacked, something, I, could, n't,...","['This', 'film', 'lacked', 'something', 'I', '..."
3,"[Sorry, everyone, ,, ,, ,, I, know, this, is, ...","['Sorry', 'everyone', 'I', 'know', 'supposed',..."
4,"[When, I, was, little, my, parents, took, me, ...","['When', 'I', 'little', 'parent', 'took', 'alo..."


In [38]:
imdb_train.describe()

,text_final_train,reviews of train
count,25000,25000
unique,24902,2
top,"['This', 'show', 'come', 'interesting', 'locat...",pos
freq,3,12500


In [19]:
imdb_train = imdb_train.drop('text_train', 1)


In [20]:
imdb_train.head()

,text_final_train
0,"['Story', 'man', 'unnatural', 'feeling', 'pig'..."
1,"['Airport', 'start', 'brand', 'new', 'luxury',..."
2,"['This', 'film', 'lacked', 'something', 'I', '..."
3,"['Sorry', 'everyone', 'I', 'know', 'supposed',..."
4,"['When', 'I', 'little', 'parent', 'took', 'alo..."


In [21]:
# Define the Target Variable
imdb_train["reviews of train"] = ['pos']*12500+['neg']*12500
imdb_train.head()

,text_final_train,reviews of train
0,"['Story', 'man', 'unnatural', 'feeling', 'pig'...",pos
1,"['Airport', 'start', 'brand', 'new', 'luxury',...",pos
2,"['This', 'film', 'lacked', 'something', 'I', '...",pos
3,"['Sorry', 'everyone', 'I', 'know', 'supposed',...",pos
4,"['When', 'I', 'little', 'parent', 'took', 'alo...",pos


In [22]:
train_X, test_X, train_y, test_y = train_test_split(imdb_train['text_final_train'],imdb_train["reviews of train"],test_size=0.3,random_state=123)


# count vectorization

In [26]:
# Extending stop words list
stop_words = stopwords.words('english')

# Adding more stop words to existing stopwords list
stop_words.extend(['aa','aah','aaniye','abj','ag'])

count_vect = CountVectorizer(stop_words=stop_words, max_features=5000)
count_vect.fit(imdb_train['text_final_train'])

train_X_count = count_vect.transform(train_X)
test_X_count = count_vect.transform(test_X)

In [64]:
print(type(count_vect))

<class 'sklearn.feature_extraction.text.CountVectorizer'>


In [28]:
print(type(train_X_count))

<class 'scipy.sparse.csr.csr_matrix'>


In [29]:
train_X_count.shape

(17500, 5000)

In [30]:
# Coverting to dense matrix and putting in a dataframe to view the Tfidf matrix
Dense_mat = train_X_count.todense()
count_Mat = pd.DataFrame(Dense_mat, columns=count_vect.get_feature_names())
# Tfidf_Mat.shape
count_Mat.head()

,abandoned,abc,ability,able,abraham,abrupt,absence,absent,absolute,absolutely,...,york,young,younger,youngest,youth,zero,zizek,zombie,zombies,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,3,0,0


In [46]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(kernel='linear')
SVM.fit(train_X_count,train_y)

# predict the labels on train dataset
pred_train1 = SVM.predict(train_X_count)

# predict the labels on validation dataset
pred_test1 = SVM.predict(test_X_count)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score on Train set -> ", accuracy_score(train_y, pred_train1)*100)
print("SVM Accuracy Score on Validation set -> ",accuracy_score(test_y, pred_test1)*100)

SVM Accuracy Score on Train set ->  98.90857142857142
SVM Accuracy Score on Validation set ->  82.92


In [63]:
print(classification_report(test_y,pred_test1,digits=4))

              precision    recall  f1-score   support

         neg     0.8311    0.8314    0.8312      3795
         pos     0.8272    0.8270    0.8271      3705

    accuracy                         0.8292      7500
   macro avg     0.8292    0.8292    0.8292      7500
weighted avg     0.8292    0.8292    0.8292      7500



# predictions on test data

In [47]:
imdb_test.shape

(25000, 1)

In [48]:
imdb_test.head()

,text_test
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."
3,Not even the Beatles could write songs everyon...
4,Brass pictures (movies is not a fitting word f...


In [49]:
imdb_test['text_test'] = [word_tokenize(entry) for entry in imdb_test['text_test']]

print(imdb_test['text_test'].head())

0    [Once, again, Mr., Costner, has, dragged, out,...
1    [This, is, an, example, of, why, the, majority...
2    [First, of, all, I, hate, those, moronic, rapp...
3    [Not, even, the, Beatles, could, write, songs,...
4    [Brass, pictures, (, movies, is, not, a, fitti...
Name: text_test, dtype: object


In [50]:
imdb_train.isnull().sum()

text_final_train    0
reviews of train    0
dtype: int64

In [51]:
# Initializing WordNetLemmatizer()
word_Lemmatized = WordNetLemmatizer()

In [52]:
for index, entry in enumerate(imdb_test['text_test']):
    #print("entry:",entry)
    
    # Declaring an Empty List to store the words that follow the rules for this step
    final_words = []
    
    for word in entry:
        #print("word:",word)
        
        
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_final = word_Lemmatized.lemmatize(word)
            final_words.append(word_final)
            
    # The final processed set of words for each iteration will be stored in 'text_train'
    imdb_test.loc[index,'text_final_test'] = str(final_words) 

In [53]:
imdb_test.head()

,text_test,text_final_test
0,"[Once, again, Mr., Costner, has, dragged, out,...","['Once', 'Costner', 'dragged', 'movie', 'far',..."
1,"[This, is, an, example, of, why, the, majority...","['This', 'example', 'majority', 'action', 'fil..."
2,"[First, of, all, I, hate, those, moronic, rapp...","['First', 'I', 'hate', 'moronic', 'rapper', 'a..."
3,"[Not, even, the, Beatles, could, write, songs,...","['Not', 'even', 'Beatles', 'could', 'write', '..."
4,"[Brass, pictures, (, movies, is, not, a, fitti...","['Brass', 'picture', 'movie', 'fitting', 'word..."


In [ ]:
imdb_test = imdb_test.drop('text_test', 1)


In [56]:
imdb_test.head()

,text_final_test
0,"['Once', 'Costner', 'dragged', 'movie', 'far',..."
1,"['This', 'example', 'majority', 'action', 'fil..."
2,"['First', 'I', 'hate', 'moronic', 'rapper', 'a..."
3,"['Not', 'even', 'Beatles', 'could', 'write', '..."
4,"['Brass', 'picture', 'movie', 'fitting', 'word..."


In [57]:
# Extending stop words list
Stop_words = stopwords.words('english')

# Adding more stop words to existing stopwords list
Stop_words.extend(['aa','aah','aaniye','abj','ag'])

count_vect_test = CountVectorizer(stop_words=stop_words, max_features=5000)
count_vect_test.fit(imdb_test['text_final_test'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=5000, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [67]:
test_X_count = count_vect.transform(imdb_test['text_final_test'])

In [68]:
predictions=SVM.predict(test_X_count)


In [69]:
predictions

array(['pos', 'neg', 'pos', ..., 'pos', 'neg', 'neg'], dtype=object)

In [71]:
imdb_test['sentiment analysis of review']=predictions

In [72]:
imdb_test.head()

,text_final_test,sentiment analysis of review
0,"['Once', 'Costner', 'dragged', 'movie', 'far',...",pos
1,"['This', 'example', 'majority', 'action', 'fil...",neg
2,"['First', 'I', 'hate', 'moronic', 'rapper', 'a...",pos
3,"['Not', 'even', 'Beatles', 'could', 'write', '...",neg
4,"['Brass', 'picture', 'movie', 'fitting', 'word...",neg
